# 準備

In [ ]:
import getpass
import os

# OpenAI API キーの設定
api_key = getpass.getpass("OpenAI API キーを入力してください: ")
os.environ["OPENAI_API_KEY"] = api_key

# 2.2.2 チャットアプリ

In [ ]:
from langchain_openai.chat_models import ChatOpenAI
from langchain_core.messages import HumanMessage

# 1 モデルの定義
llm = ChatOpenAI(model="gpt-4o-mini")

history = []
n = 10
for i in range(10):
    user_input = input("ユーザ入力: ")
    if user_input == "exit":
        break
    # 2 HumanMessage の作成と表示
    human_message = HumanMessage(user_input)
    human_message.pretty_print()
    # 3 会話履歴の追加
    history.append(HumanMessage(user_input))
    # 4 応答の作成と表示
    ai_message = llm.invoke(history)
    ai_message.pretty_print()
    # 5 会話履歴の追加
    history.append(ai_message)

# 2.2.3 翻訳アプリ

In [ ]:
from langchain_openai.chat_models import ChatOpenAI
from langchain_core.prompts import PromptTemplate

llm = ChatOpenAI(model="gpt-4o-mini")

# 1 テンプレートの作成
TRANSLATION_PROMPT = """\
以下の文章を {language} に翻訳し、翻訳結果のみを返してください。
{source_text}
"""
prompt = PromptTemplate.from_template(TRANSLATION_PROMPT)

# 2 Runnable の作成
runnable = prompt | llm

language = "日本語"
source_text = """\
cogito, ergo sum
"""

# 3 Runnable の実行と結果の表示
response = runnable.invoke(dict(language=language, source_text=source_text))
response.pretty_print()

# 2.2.4 テーブル作成アプリ

In [ ]:
from langchain_core.tools import tool
from langchain_core.pydantic_v1 import BaseModel, Field
from langchain_openai.chat_models import ChatOpenAI
import csv


# 1 入力形式の定義
class CSVSaveToolInput(BaseModel):
    filename: str = Field(description="ファイル名")
    csv_text: str = Field(description="CSVのテキスト")


# 2 ツール本体の定義
@tool("csv-save-tool", args_schema=CSVSaveToolInput)
def csv_save(filename: str, csv_text: str) -> bool:
    """CSV テキストをファイルに保存する"""
    # parse CSV text
    try:
        rows = list(csv.reader(csv_text.splitlines()))
    except Exception as e:
        return False

    # save to file
    with open(filename, "w") as f:
        writer = csv.writer(f)
        writer.writerows(rows)

    return True

In [ ]:
# 3 ツールを LLM に紐づける
llm = ChatOpenAI(model="gpt-4o-mini")
tools = [csv_save]
llm_with_tool = llm.bind_tools(tools=tools, tool_choice="csv-save-tool")

TABLE_PROMPT = """\
{user_input}

結果は CSV ファイルに保存してください。ただし、ファイル名は上記の内容から適切に決定してください。
"""
prompt = PromptTemplate.from_template(TABLE_PROMPT)
get_tool_args = lambda x: x.tool_calls[0]

# 4 Runnable の作成
runnable = prompt | llm_with_tool | get_tool_args | csv_save

user_input = "フィボナッチ数列の番号と値を10番目まで表にまとめて、CSV ファイルに保存してください。"

# 5 Runnable の実行と結果の確認
response = runnable.invoke(dict(user_input=user_input))
print(response)

# 2.2.5 Plan-and-Solve

In [ ]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-4o-mini")

In [ ]:
from pydantic import BaseModel, Field


# ツール入力形式の定義
class ActionItem(BaseModel):
    action_name: str = Field(description="アクション名")
    action_description: str = Field(description="アクションの詳細")


class Plan(BaseModel):
    """アクションプランを格納する"""

    problem: str = Field(description="問題の説明")
    actions: list[ActionItem] = Field(description="実行すべきアクションリスト")


class ActionResult(BaseModel):
    """実行時の考えと結果を格納する"""

    thoughts: str = Field(description="検討内容")
    result: str = Field(description="結果")

In [ ]:
# 単一アクションの実行
from langchain_openai.output_parsers.tools import PydanticToolsParser
from langchain_core.prompts import PromptTemplate


ACTION_PROMPT = """\
問題をアクションプランに分解して解いています。
これまでのアクションの結果と、次に行うべきアクションを示すので、実際にアクションを実行してその結果を報告してください。
# 問題
{problem}
# アクションプラン
{action_items}
# これまでのアクションの結果
{action_results}
# 次のアクション
{next_action}
"""

llm_action = llm.bind_tools([ActionResult], tool_choice="ActionResult")
action_parser = PydanticToolsParser(tools=[ActionResult], first_tool_only=True)
plan_parser = PydanticToolsParser(tools=[Plan], first_tool_only=True)

action_prompt = PromptTemplate.from_template(ACTION_PROMPT)
action_runnable = action_prompt | llm_action | action_parser

In [ ]:
# プランに含まれるアクションを実行するループ
from langchain_core.messages import AIMessage


def action_loop(action_plan: Plan):
    problem = action_plan.problem
    actions = action_plan.actions

    action_items = "\n".join(["* " + action.action_name for action in actions])
    action_results = []
    action_results_str = ""
    for i, action in enumerate(actions):
        print("=" * 20)
        print(f"[{i+1}/{len(actions)}]以下のアクションを実行します。")
        print(action.action_name)

        next_action = f"* {action.action_name}  \n{action.action_description}"
        response = action_runnable.invoke(
            dict(
                problem=problem,
                action_items=action_items,
                action_results=action_results_str,
                next_action=next_action,
            )
        )
        action_results.append(response)
        action_results_str += f"* {action.action_name}  \n{response.result}\n"
        print("-" * 10 + "検討内容" + "-" * 10)
        print(response.thoughts)
        print("-" * 10 + "結果" + "-" * 10)
        print(response.result)

    return AIMessage(action_results_str)

In [ ]:
# Plan-and-Solve を行うか否かの分岐
def route(ai_message: AIMessage):
    if ai_message.response_metadata["finish_reason"] == "tool_calls":
        return plan_parser | action_loop
    else:
        return ai_message

In [ ]:
# 全体を通した Runnable 作成
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.messages import SystemMessage

PLAN_AND_SOLVE_PROMPT = """\
ユーザの質問が複雑な場合は、アクションプランを作成し、その後に1つずつ実行する Plan-and-Solve 形式をとります。
これが必要と判断した場合は、Plan ツールによってアクションプランを保存してください。
"""
system_prompt = SystemMessage(PLAN_AND_SOLVE_PROMPT)
chat_prompt = ChatPromptTemplate.from_messages(
    [system_prompt, MessagesPlaceholder(variable_name="history")]
)

llm_plan = llm.bind_tools(tools=[Plan])
planning_runnable = chat_prompt | llm_plan | route

In [ ]:
# チャット部分の作成
history = []
n = 10
for i in range(10):
    user_input = input("ユーザ入力: ")
    if user_input == "exit":
        break
    # 1 HumanMessage の作成と表示
    human_message = HumanMessage(user_input)
    human_message.pretty_print()
    # 2 会話履歴の追加
    history.append(HumanMessage(user_input))
    # 3 応答の作成と表示
    ai_message = planning_runnable.invoke(dict(history=history))
    ai_message.pretty_print()
    # 4 会話履歴の追加
    history.append(ai_message)

In [ ]:
# 出力例

# ================================ Human Message =================================
#
# ある製造工場では、1時間に200個の部品が生産されます。工場は1日8時間稼働し、1週間に5日間営業しています。生産された部品のうち5%は品質不良で廃棄されます。この工場では1ヶ月（4週間）に品質不良で廃棄される部品の総数を求めなさい。
# ====================
# [1/4]以下のアクションを実行します。
# 部品の1日の生産量を求める
# ----------検討内容----------
# 部品の1日の生産量を求めた結果、1日の生産量は1600個である。
# ----------結果----------
# 部品の1日の生産量は1600個である。
# ====================
# [2/4]以下のアクションを実行します。
# 部品の1週間の生産量を求める
# ----------検討内容----------
# 1週間の生産量を求めた。1日の生産量1600個に5営業日を掛けて計算した結果、8000個となった。この結果を記録する。
# ----------結果----------
# 部品の1週間の生産量は8000個である。
# ====================
# [3/4]以下のアクションを実行します。
# 部品の1ヶ月の生産量を求める
# ----------検討内容----------
# 部品の1ヶ月の生産量は8000個 × 4週間 = 32000個と計算した。
# ----------結果----------
# 部品の1ヶ月の生産量は32000個である。
# ====================
# [4/4]以下のアクションを実行します。
# 品質不良で廃棄される部品の数を求める
# ----------検討内容----------
# 品質不良で廃棄される部品の数を求めるために、1ヶ月の生産量32000個に5%を掛け算して1600個を算出する。
# ----------結果----------
# 品質不良で廃棄される部品の数は1600個である。
# ================================== Ai Message ==================================
#
# * 部品の1日の生産量を求める
# 部品の1日の生産量は1600個である。
# * 部品の1週間の生産量を求める
# 部品の1週間の生産量は8000個である。
# * 部品の1ヶ月の生産量を求める
# 部品の1ヶ月の生産量は32000個である。
# * 品質不良で廃棄される部品の数を求める
# 品質不良で廃棄される部品の数は1600個である。